In [1]:
import rasterio
from rasterio.plot import show
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
from skimage import io
import torchvision.transforms as T
from torchmetrics import F1Score
import sys
sys.path.append('../Models/Networks')
from U_Net import *

C:\Users\mdomi\miniconda3\envs\pytorch\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


## Load the model

In [6]:
model = torch.load('../Models/Experiments/U_Net_TargetOnlyTNZ.pt')

In [7]:
def F1(pred, gt):

    cl = pred - gt
    
    FN = np.sum(cl == -1)
    FP = np.sum(cl == 1)
    TN = np.sum(pred[((pred == 0)&(cl == 0))[0]])
    TP = np.sum(pred[((pred == 1)&(cl == 0))[0]])

    if TP == 0 :
        F1 = 0
    else:
        prec = TP/(TP+FP)
        recall = TP/(TP+FN)

        F1 = 2*(prec*recall)/(prec+recall)

    return F1

In [ ]:
country = 'IvoryCoast'

img_folder = '../Data/Planet/'+country+'/'

F1s = []
OAs = []

F1 = F1Score(task = 'binary')

for idx in range(int(len(os.listdir(img_folder))/2)):    
    
    conversion = T.ToTensor()
    
    img = io.imread(fname = img_folder + '/Cropped' + country + 'StudyArea_{:05d}'.format(idx) + '.tif').astype(np.float32)
    gt = io.imread(fname = img_folder + '/Cropped' + country + 'GT_{:05d}'.format(idx) + '.tif').astype(np.float32)
    
    # img = img/np.max(np.max(img, 0), 0)
    img = img/np.max(img)
    
    img = conversion(img).float()[None, :, :, :]

    pred = model(img.to('cuda'))
    pred_ = pred[0].max(0)[1].to('cpu')
    
    gt = conversion(gt)[0].to('cpu')

    f1 = F1(pred_, gt).numpy()

    F1s.append(f1)
    OAs.append(np.mean((pred_ == gt).numpy()))
    
    if f1 > 0.8:
        fig, ax = plt.subplots(1,1,figsize = (6,6))
        
        ax.imshow(np.transpose(img[0,[2,1,0],:,:], (1,2,0)))
        
        ax.imshow(pred_, cmap = 'gray', alpha = 0.15)

In [ ]:
np.mean(F1s)

## For Tanzania as the Target Domain

|**Model** | **F1-score** | **Overall accuracy** |
| ----- | --------| --- |
| *Source-only (CIV)* | 0.05 | 0.61 |
| *Target-only (TNZ)* | 0.32 | 0.58 |  

## For Ivory Coast as the Target Domain
|**Model** | **F1-score** | **Overall Accuracy** | 
| ----- | --------| -- |
| *Source-only (TNZ)* |  |  |
| *Target-only (CIV)* |  |  |